In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from stable_audio_tools.models.autoencoders import AudioConditionalVariationalAutoEncoder
from stable_audio_tools.models import create_model_from_config
from stable_audio_tools.models.utils import copy_state_dict
from stable_audio_tools.data.utils import PadCrop_Normalized_T, Stereo, Mono

import torch
import torch.nn.functional as F
import torchaudio
from torchaudio import transforms as T
from pathlib import Path
from IPython.display import Audio

In [18]:
spec.shape

torch.Size([2, 1025, 14])

### Load Model

In [3]:
loaded = torch.load("/home/sake/userdata/sake/stable-audio-cvae/oneshot-drums-conditional-vae/fky8t0jw/checkpoints/epoch=80-step=50000.ckpt")

/tmp/ipykernel_1103247/449672252.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded = torch.load("/home/sake/userdata/sake/stable-audio-cvae/oneshot-drums-conditiona

In [4]:
loaded.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'MixedPrecisionPlugin', 'model_config'])

In [5]:
state_dict = {}
for key in loaded['state_dict']:
  if "autoencoder" in key:
    keyy = key.replace("autoencoder.", "")
    state_dict[keyy] = loaded['state_dict'][key]

In [6]:
model = create_model_from_config(loaded['model_config'])
copy_state_dict(model, state_dict)
model = model.eval()
model = model.to("cuda")

/home/sake/.local/share/virtualenvs/stable-audio-cvae-diFMr3O8/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


### Load Data

In [7]:
pad_crop = PadCrop_Normalized_T(n_samples=65536, sample_rate=44100)
to_stereeo = Stereo()

In [8]:
# audio_fn = Path("/home/sake/userdata/sake/oneshot_data/percussion/Medicated OHat 6.wav")
# audio_fn = Path("/home/sake/userdata/sake/oneshot_data/snare/07_Snare_05_SP.wav")
audio_fn = Path("/home/sake/userdata/sake/oneshot_data/kick/001_50S1.wav")
wav, sr = torchaudio.load(audio_fn)
if sr != 44100:
  resample_tf = T.Resample(sr, 44100)
  wav = resample_tf(wav)
wav, t_start, t_end, seconds_start, seconds_total, padding_mask = pad_crop(wav)
wav = wav.clamp(-1, 1)
wav = to_stereeo(wav)

In [9]:
wav.shape

torch.Size([2, 65536])

- 'kick': 0, 
- 'snare': 1, 
- 'hat': 2,
- 'clap': 3,
- 'percussion': 4
- None: 5 ?

In [10]:
og_labels = torch.tensor([0])
tgt_labels = torch.tensor([1])
none_labels = torch.tensor([5])
wav.shape, og_labels, tgt_labels

(torch.Size([2, 65536]), tensor([0]), tensor([1]))

### Reconstruct

In [11]:
with torch.no_grad():
  latents = model.encode(wav.unsqueeze(0).to("cuda"), condition=og_labels.unsqueeze(0).to("cuda"))
  twist_rec = model.decode(latents, condition=tgt_labels.unsqueeze(0).to("cuda"))

In [14]:
with torch.no_grad():
  latents = model.encode(wav.unsqueeze(0).to("cuda"), condition=none_labels.unsqueeze(0).to("cuda"))
  none_tgt_rec = model.decode(latents, condition=tgt_labels.unsqueeze(0).to("cuda"))

In [15]:
with torch.no_grad():
  latents = model.encode(wav.unsqueeze(0).to("cuda"), condition=tgt_labels.unsqueeze(0).to("cuda"))
  tgt_rec = model.decode(latents, condition=tgt_labels.unsqueeze(0).to("cuda"))

In [16]:
og_rec = model(wav.unsqueeze(0).to("cuda"), og_labels.unsqueeze(0).to("cuda"))

In [17]:
none_rec = model(wav.unsqueeze(0).to("cuda"), none_labels.unsqueeze(0).to("cuda"))

In [18]:
Audio(wav[0].numpy(), rate=44100)

In [19]:
Audio(og_rec[0].cpu().detach().numpy(), rate=44100)

In [12]:
Audio(twist_rec[0].cpu().detach().numpy(), rate=44100)

In [21]:
Audio(tgt_rec[0].cpu().detach().numpy(), rate=44100)

In [22]:
Audio(none_tgt_rec[0].cpu().detach().numpy(), rate=44100)

In [23]:
Audio(none_rec[0].cpu().detach().numpy(), rate=44100)

In [24]:
latents.shape

torch.Size([1, 64, 64])

### Sampling

In [13]:
distribution_stats = torch.load("distribution_stats-epoch=80-step=50000.pt")

/tmp/ipykernel_1103247/1206692969.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  distribution_stats = torch.load("distribution_stats-epoch=80-step=50000.pt")


In [15]:
distribution_stats

{'kick': {'mean': tensor([[-9.0142e-02, -1.1130e-01, -7.9726e-02,  ...,  6.5950e-02,
            8.5308e-02,  2.2713e-02],
          [ 1.3297e+00,  1.1377e+00,  2.1310e+00,  ..., -1.3060e+00,
           -1.3937e+00, -1.4890e+00],
          [ 5.5085e-02,  1.7205e-02, -1.2548e-02,  ..., -1.1136e-02,
           -2.3410e-02, -1.0421e-02],
          ...,
          [-1.1462e-01, -8.1049e-02, -1.0918e-01,  ..., -3.1445e-02,
           -5.8746e-02, -1.1104e-01],
          [ 5.6054e-02,  1.2835e-01,  1.2147e-01,  ...,  1.4799e-03,
            3.3950e-02, -6.6385e-03],
          [ 6.7787e-04, -6.4375e-03, -2.0005e-02,  ...,  1.2598e-02,
            5.8227e-03, -2.9791e-03]]),
  'std': tensor([[0.9765, 0.9820, 0.9594,  ..., 1.0058, 1.0088, 0.9847],
          [2.7041, 2.5101, 2.2232,  ..., 0.4890, 0.5068, 0.6243],
          [0.9348, 0.8799, 0.8358,  ..., 1.0253, 0.9912, 0.9450],
          ...,
          [0.8859, 0.9090, 0.9274,  ..., 0.9969, 0.9626, 0.9559],
          [0.9284, 0.9195, 0.9055,  ...

In [31]:
def sample_latent_from_stats(instrument, distribution_stats, device="cuda"):
  """
  Sample latent vector from distribution statistics for a given instrument
  Args:
    instrument: str, one of ['kick', 'snare', 'hat', 'clap', 'percussion', 'total']
    distribution_stats: dict containing mean and std for each instrument
    device: torch device
  Returns:
    sampled latent vector of shape [1, 64, 64]
  """
  if instrument not in distribution_stats:
    raise ValueError(f"Instrument {instrument} not found in distribution stats")
    
  mean = distribution_stats[instrument]['mean'].to(device)
  std = distribution_stats[instrument]['std'].to(device)
  
  # Sample from standard normal and scale by mean/std
  z = torch.randn(1, 64, 64, device=device)
  sampled = z * std + mean
  
  return sampled


- 'kick': 0, 
- 'snare': 1, 
- 'hat': 2,
- 'clap': 3,
- 'percussion': 4
- None: 5 ?

In [52]:
kick_labels = torch.tensor([0])
snare_labels = torch.tensor([1])
hat_labels = torch.tensor([2])
clap_labels = torch.tensor([3])
percussion_labels = torch.tensor([4])
none_labels = torch.tensor([5])

In [72]:
# Sample random latents from standard normal distribution
# sampled_latents = 0.1 * torch.randn(1, 64, 64, device="cuda")

# 각 악기의 latent 분포에서 샘플 ['kick', 'snare', 'hat', 'clap', 'percussion', 'total']
sampled_latents = sample_latent_from_stats("snare", distribution_stats, device="cuda")


In [73]:

# Generate audio from sampled latents using different conditions
none_gen = model.decode(sampled_latents, none_labels.unsqueeze(0).to("cuda"))
kick_gen = model.decode(sampled_latents, kick_labels.unsqueeze(0).to("cuda")) 
snare_gen = model.decode(sampled_latents, snare_labels.unsqueeze(0).to("cuda"))
hat_gen = model.decode(sampled_latents, hat_labels.unsqueeze(0).to("cuda"))
clap_gen = model.decode(sampled_latents, clap_labels.unsqueeze(0).to("cuda"))
percussion_gen = model.decode(sampled_latents, percussion_labels.unsqueeze(0).to("cuda"))

In [74]:
Audio(none_gen[0].cpu().detach().numpy(), rate=44100)

In [75]:
Audio(kick_gen[0].cpu().detach().numpy(), rate=44100)

In [76]:
Audio(snare_gen[0].cpu().detach().numpy(), rate=44100)

In [77]:
Audio(hat_gen[0].cpu().detach().numpy(), rate=44100)

In [78]:
Audio(clap_gen[0].cpu().detach().numpy(), rate=44100)

In [79]:
Audio(percussion_gen[0].cpu().detach().numpy(), rate=44100)

### Get distribution stats from latents

In [23]:
lo = torch.load("/home/sake/userdata/sake/stable-audio-cvae/latents-epoch=80-step=50000.pt")

/tmp/ipykernel_1103247/2273288854.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lo = torch.load("/home/sake/userdata/sake/stable-audio-cvae/latents-epoch=80-step=50000

In [24]:
lo.keys()

dict_keys(['kick', 'snare', 'hat', 'clap', 'percussion'])

In [27]:
kick_latents = []
snare_latents = []
hat_latents = []
clap_latents = []
percussion_latents = []
total_latents = []

for k in lo.keys():
  for i in lo[k].keys():
    if k == 'kick':
      kick_latents.append(lo[k][i]['embedding'])
    elif k == 'snare':
      snare_latents.append(lo[k][i]['embedding'])
    elif k == 'hat':
      hat_latents.append(lo[k][i]['embedding'])
    elif k == 'clap':
      clap_latents.append(lo[k][i]['embedding'])
    elif k == 'percussion':
      percussion_latents.append(lo[k][i]['embedding'])
    total_latents.append(lo[k][i]['embedding'])

kick_mean = torch.mean(torch.stack(kick_latents, dim=0), dim=0)
snare_mean = torch.mean(torch.stack(snare_latents, dim=0), dim=0)
hat_mean = torch.mean(torch.stack(hat_latents, dim=0), dim=0)
clap_mean = torch.mean(torch.stack(clap_latents, dim=0), dim=0)
percussion_mean = torch.mean(torch.stack(percussion_latents, dim=0), dim=0)
total_mean = torch.mean(torch.stack(total_latents, dim=0), dim=0)

kick_std = torch.std(torch.stack(kick_latents, dim=0), dim=0)
snare_std = torch.std(torch.stack(snare_latents, dim=0), dim=0)
hat_std = torch.std(torch.stack(hat_latents, dim=0), dim=0)
clap_std = torch.std(torch.stack(clap_latents, dim=0), dim=0)
percussion_std = torch.std(torch.stack(percussion_latents, dim=0), dim=0)
total_std = torch.std(torch.stack(total_latents, dim=0), dim=0)


In [28]:
distribution_stats = {
  'kick': {
    'mean': kick_mean,
    'std': kick_std
  },
  'snare': {
    'mean': snare_mean,
    'std': snare_std
  },
  'hat': {
    'mean': hat_mean,
    'std': hat_std
  },
  'clap': {
    'mean': clap_mean,
    'std': clap_std
  },
  'percussion': {
    'mean': percussion_mean,
    'std': percussion_std
  },
  'total': {
    'mean': total_mean,
    'std': total_std
  }
}


In [29]:
torch.save(distribution_stats, "distribution_stats-epoch=80-step=50000.pt")